# Circle

In [1]:
%%capture
!pip install iblm --upgrade

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git 

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.1


In [2]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier


import os
#os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [3]:
n = 600

df = pd.read_csv(f'../data/circle/circle_{n}_train.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']
print(len(x_train))

300


In [4]:
x_train

,Feature_1,Feature_2
0,0.656472,0.480717
1,0.772840,0.064924
2,-0.183736,-0.791791
3,0.475243,-0.664004
4,0.705191,0.466519
...,...,...
295,0.716764,0.663982
296,-0.155343,0.921014
297,-0.964970,-0.258616
298,0.512335,-0.937646


In [12]:
llm_model_name = 'gpt-4'

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model_name=llm_model_name, params=params)

### Training

In [30]:
file_path = '../models/circle/'

print(f'Number of data:{len(x_train)}')
model = iblm.fit(x_train, y_train, model_name = 'circle', file_path=file_path)

Number of data:300
> Start of model creating.
Tokens Used: 7161
	Prompt Tokens: 6988
	Completion Tokens: 173
Successful Requests: 1
Total Cost (USD): $0.22002


In [32]:
# Code of the model created
print(model)

import numpy as np
def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.

        feature_1 = row['Feature_1']
        feature_2 = row['Feature_2']

        # Calculate the distance from the origin (0, 0)
        distance = np.sqrt(feature_1**2 + feature_2**2)

        # Normalize the distance to range [0, 1]
        normalized_distance = distance / np.sqrt(2)

        # Calculate the probability of the target being 1
        y = 1 - normalized_distance

        # Do not change the code after this point.
        output.append(y)
    return np.array(output)


### Prediction

In [33]:
df = pd.read_csv(f'../data/circle/circle_{n}_test.csv')
x_test = df.drop('Target', axis=1)
y_test = df['Target']

In [34]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [35]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.51
Precision: 1.0
Recall: 0.02
F1 score: 0.0392156862745098
ROC-AUC: 0.9991111111111111
